<a href="https://colab.research.google.com/github/tom5610/ml-practices-using-books/blob/main/practical-ml-vision-book/02_ml_models/02b_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>